In [4]:
%matplotlib inline                      

import seaborn as sns                   # for beautiful plots
from scipy import stats                 # to calculate mode, skew and ANOVA, etc.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm, skew #for some statistics
plt.rcParams['figure.figsize'] = (10, 8)
# importing alll the necessary packages to use the various classification algorithms
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.metrics import mean_squared_error #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [25]:
train = pd.read_csv('../house price/train.csv')
test = pd.read_csv('../house price/test.csv')
Id = test.Id
target = train.SalePrice
data = pd.read_csv('../house price/004_data.csv')

In [16]:
nums = [f for f in train.columns if train.dtypes[f] != 'object']
nums.remove('SalePrice')
nums.remove('Id')
cats = [f for f in train.columns if train.dtypes[f] == 'object']

In [26]:
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)
train['SalePrice'] = np.log(train['SalePrice'])

In [166]:
alldata = pd.concat([train, test], sort=False)
alldata

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,12.247694
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,12.109011
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,12.317167
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,11.849398
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,12.429216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
1455,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
1456,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
1457,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [167]:
group = train.groupby('MSZoning').agg({'SalePrice':['mean']})
group.columns = ['mean_MSZoning_Price']
group.reset_index(inplace=True)
alldata = pd.merge(alldata, group, on=['MSZoning'], how='left')
alldata

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,mean_MSZoning_Price
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,0,2,2008,WD,Normal,12.247694,12.085885
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,NaN,NaN,NaN,0,5,2007,WD,Normal,12.109011,12.085885
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,NaN,NaN,0,9,2008,WD,Normal,12.317167,12.085885
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,11.849398,12.085885
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,NaN,NaN,0,12,2008,WD,Normal,12.429216,12.085885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN,11.692893
2915,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN,11.692893
2916,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN,12.085885
2917,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN,12.085885


## カテゴリ変数と数値変数

In [168]:
def cat_group_feature(df, train_df, cat_f, agg):
    for c in cat_f:
        group = train_df.groupby(c).agg({'SalePrice':[agg]})
        group.columns = [agg + '_' + c + '_Price']
        group.reset_index(inplace=True)
        df = df.merge(group, on=[c], how='left')
    return df

In [169]:
alldata = cat_group_feature(alldata, train, cats, 'mean')


In [170]:
alldata

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,mean_GarageType_Price,mean_GarageFinish_Price,mean_GarageQual_Price,mean_GarageCond_Price,mean_PavedDrive_Price,mean_PoolQC_Price,mean_Fence_Price,mean_MiscFeature_Price,mean_SaleType_Price,mean_SaleCondition_Price
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,12.160778,12.172672,12.068539,12.070483,12.059895,NaN,NaN,NaN,11.991061,12.005034
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,12.160778,12.172672,12.068539,12.070483,12.059895,NaN,NaN,NaN,11.991061,12.005034
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,12.160778,12.172672,12.068539,12.070483,12.059895,NaN,NaN,NaN,11.991061,12.005034
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,11.765643,11.818974,12.068539,12.070483,12.059895,NaN,NaN,NaN,11.991061,11.788774
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,12.160778,12.172672,12.068539,12.070483,12.059895,NaN,NaN,NaN,11.991061,12.005034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,NaN,12.059895,NaN,NaN,NaN,11.991061,12.005034
2915,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,11.587994,11.818974,12.068539,12.070483,12.059895,NaN,NaN,NaN,11.991061,11.788774
2916,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,11.765643,11.818974,12.068539,12.070483,12.059895,NaN,NaN,NaN,11.991061,11.788774
2917,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,NaN,12.059895,NaN,11.850596,11.868387,11.991061,12.005034


## 数値変数と数値変数

In [171]:
def num_mult(df, num_features):
    for i in num_features:
        for j in num_features:
            if i != j:
                df[i + ' * ' + j] = df[i] * df[j]
    return df

In [172]:
num_mult(alldata, nums)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,YrSold * GarageCars,YrSold * GarageArea,YrSold * WoodDeckSF,YrSold * OpenPorchSF,YrSold * EnclosedPorch,YrSold * 3SsnPorch,YrSold * ScreenPorch,YrSold * PoolArea,YrSold * MiscVal,YrSold * MoSold
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,4016.0,1100384.0,0,122488,0,0,0,0,0,4016
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,4014.0,923220.0,598086,0,0,0,0,0,0,10035
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,4016.0,1220864.0,0,84336,0,0,0,0,0,18072
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,6018.0,1287852.0,0,70210,545632,0,0,0,0,4012
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,6024.0,1678688.0,385536,168672,0,0,0,0,0,24096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0.0,0.0,0,0,0,0,0,0,0,12036
2915,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,2006.0,573716.0,0,48144,0,0,0,0,0,8024
2916,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,4012.0,1155456.0,950844,0,0,0,0,0,0,18054
2917,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0.0,0.0,160480,64192,0,0,0,0,1404200,14042


In [173]:
def num_div(df, num_features):
    for i in num_features:
        for j in num_features:
            if i != j:
                df[i + ' / ' + j] = df[i] / df[j]
    df = df.replace(np.inf, np.nan)
    return df

In [174]:
num_div(alldata, nums)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,YrSold / GarageCars,YrSold / GarageArea,YrSold / WoodDeckSF,YrSold / OpenPorchSF,YrSold / EnclosedPorch,YrSold / 3SsnPorch,YrSold / ScreenPorch,YrSold / PoolArea,YrSold / MiscVal,YrSold / MoSold
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,1004.000000,3.664234,NaN,32.918033,NaN,NaN,NaN,NaN,NaN,1004.000000
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,1003.500000,4.363043,6.734899,NaN,NaN,NaN,NaN,NaN,NaN,401.400000
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,1004.000000,3.302632,NaN,47.809524,NaN,NaN,NaN,NaN,NaN,223.111111
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,668.666667,3.124611,NaN,57.314286,7.375,NaN,NaN,NaN,NaN,1003.000000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,669.333333,2.401914,10.458333,23.904762,NaN,NaN,NaN,NaN,NaN,167.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334.333333
2915,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,2006.000000,7.013986,NaN,83.583333,NaN,NaN,NaN,NaN,NaN,501.500000
2916,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,1003.000000,3.482639,4.232068,NaN,NaN,NaN,NaN,NaN,NaN,222.888889
2917,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,25.075000,62.687500,NaN,NaN,NaN,NaN,2.865714,286.571429


In [175]:
alldata = alldata.replace(np.inf, np.nan)

In [176]:
alldata

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,YrSold / GarageCars,YrSold / GarageArea,YrSold / WoodDeckSF,YrSold / OpenPorchSF,YrSold / EnclosedPorch,YrSold / 3SsnPorch,YrSold / ScreenPorch,YrSold / PoolArea,YrSold / MiscVal,YrSold / MoSold
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,1004.000000,3.664234,NaN,32.918033,NaN,NaN,NaN,NaN,NaN,1004.000000
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,1003.500000,4.363043,6.734899,NaN,NaN,NaN,NaN,NaN,NaN,401.400000
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,1004.000000,3.302632,NaN,47.809524,NaN,NaN,NaN,NaN,NaN,223.111111
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,668.666667,3.124611,NaN,57.314286,7.375,NaN,NaN,NaN,NaN,1003.000000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,669.333333,2.401914,10.458333,23.904762,NaN,NaN,NaN,NaN,NaN,167.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334.333333
2915,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,2006.000000,7.013986,NaN,83.583333,NaN,NaN,NaN,NaN,NaN,501.500000
2916,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,1003.000000,3.482639,4.232068,NaN,NaN,NaN,NaN,NaN,NaN,222.888889
2917,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,25.075000,62.687500,NaN,NaN,NaN,NaN,2.865714,286.571429
